In [27]:
###Logistic Regression & RForests

from collections import Counter
import pandas as pd
import numpy as np

#Handle Imbalance Data
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import smote_variants as sv

#ML ALgorithims
from matplotlib import pyplot
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import RandomizedSearchCV

In [2]:
#Reading our data

train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

/Users/leotsang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['player_name', 'school', 'conference', 'GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'rec-rk', 'ast/tov', 'rimmade',
       'rimmade + rimmiss', 'midmade', 'midmade + midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+mismiss)', 'dunksmade',
       'dunksmiss + dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'pos', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'ht_in',
       'drafted', 'yr_cat', 'ATH', 'GP_adj', 'BBIQ'],
      dtype='object')

In [4]:
#stats to look at

per_stats = ['Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FT_per', '2P_per', '3P_per', 
             'blk_per', 'stl_per', 'GP', 'ht_in', 'yr_cat',  'ATH', 'BBIQ']
box_stats = ['GP', 'mp', 'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg','oreb', 'dreb', 'treb',
             'ast', 'stl', 'blk', 'ht_in', 'yr_cat', 'FTA', 'FTM', 'ftr',  'ATH', 'BBIQ']
adv_stats = ['GP', 'mp', 'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'oreb', 'dreb', 'treb',
             'ast', 'stl', 'blk', 'ht_in', 'yr_cat', 'bpm', 'obpm', 'dbpm', 'FTA', 'FTM', 'ftr' ,'ATH', 'BBIQ']

per_adv = ['Min_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FT_per', '2P_per', '3P_per', 
             'blk_per', 'stl_per', 'GP', 'ht_in', 'yr_cat', 'bpm', 'obpm', 'dbpm', 'ATH', 'BBIQ']

all_stats = ['GP', 'Min_per', 'ORtg', 'usg',
       'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM',
       'FTA', 'FT_per', '2PM', '2PA', '2P_per', '3PM', '3PA', '3P_per',
       'blk_per', 'stl_per', 'ftr', 'porpag', 'adjoe',
       'pfr','year', 'rec-rk', 'ast/tov', 
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', '2PA_pg', '2PM_pg', '3PA_pg', '3PM_pg', 'yr_cat', 'ht_in']

In [5]:
#delete all missing values
train = train.dropna(axis=0, subset=per_stats)
train = train.dropna(axis=0, subset=box_stats)
train = train.dropna(axis=0, subset=adv_stats)

#delete mssing value in test set as well

test = test.dropna(axis=0, subset=per_stats)
test = test.dropna(axis=0, subset=box_stats)
test = test.dropna(axis=0, subset=adv_stats)

In [6]:
#set our 2 test-set aside
test_2018 = test[test['year']==2018]
test_2019 = test[test['year']==2019]

In [17]:
tests = [per_stats, box_stats, adv_stats, per_adv] #all_stats
year = [2011, 2010, 2012, 2013, 2014, 2015, 2016, 2017]

In [43]:
###Ensemble, undersample
pred_yr = []
sample = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2]

for j in sample:
    print('#######{} SAMPLES######'.format(j))
    
    for i in year:

        train_yr = train[train['year']==i]
        model = XGBClassifier()
        over = SMOTE(sampling_strategy=j)
        under = RandomUnderSampler(sampling_strategy=j)
        steps = [('o', over), ('u', under)]
        pipeline = Pipeline(steps=steps)
        X, y = pipeline.fit_resample(train_yr[adv_stats], train_yr['drafted'])
        model.fit(X, y)
        y_pred = model.predict(test_2019[adv_stats])
        predictions = [round(value) for value in y_pred]
        pred_yr.append(predictions)

    bag_predictions = sum(np.array(pred_yr))
    bag_y = []

    for i in bag_predictions:
        if i > 4:
            bag_y.append(1)
        else:
            bag_y.append(0)

    f1 = f1_score(test_2019['drafted'], bag_y)
    recall = recall_score(test_2019['drafted'], bag_y)
    precision = precision_score(test_2019['drafted'], bag_y)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

#######0.025 SAMPLES######
F1: 0.3855421686746988 recall: 0.3137254901960784 precision: 0.5 predictions: 36.0
#######0.05 SAMPLES######
F1: 0.423728813559322 recall: 0.49019607843137253 precision: 0.373134328358209 predictions: 50.0
#######0.075 SAMPLES######
F1: 0.40579710144927533 recall: 0.5490196078431373 precision: 0.3218390804597701 predictions: 70.0
#######0.1 SAMPLES######
F1: 0.40718562874251496 recall: 0.6666666666666666 precision: 0.29310344827586204 predictions: 76.0
#######0.125 SAMPLES######
F1: 0.40659340659340665 recall: 0.7254901960784313 precision: 0.2824427480916031 predictions: 68.0
#######0.15 SAMPLES######
F1: 0.40796019900497515 recall: 0.803921568627451 precision: 0.2733333333333333 predictions: 86.0
#######0.175 SAMPLES######
F1: 0.3744292237442922 recall: 0.803921568627451 precision: 0.24404761904761904 predictions: 83.0
#######0.2 SAMPLES######
F1: 0.351931330472103 recall: 0.803921568627451 precision: 0.22527472527472528 predictions: 89.0


In [44]:
###Ensemble, undersample
pred_yr = []
sample = [0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2]

for j in sample:
    print('#######{} SAMPLES######'.format(j))
    
    for i in year:

        train_yr = train[train['year']==i]
        model = AdaBoostClassifier()
        over = SMOTE(sampling_strategy=j)
        under = RandomUnderSampler(sampling_strategy=j)
        steps = [('o', over), ('u', under)]
        pipeline = Pipeline(steps=steps)
        X, y = pipeline.fit_resample(train_yr[adv_stats], train_yr['drafted'])
        model.fit(X, y)
        y_pred = model.predict(test_2019[adv_stats])
        predictions = [round(value) for value in y_pred]
        pred_yr.append(predictions)

    bag_predictions = sum(np.array(pred_yr))
    bag_y = []

    for i in bag_predictions:
        if i > 4:
            bag_y.append(1)
        else:
            bag_y.append(0)

    f1 = f1_score(test_2019['drafted'], bag_y)
    recall = recall_score(test_2019['drafted'], bag_y)
    precision = precision_score(test_2019['drafted'], bag_y)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

#######0.025 SAMPLES######
F1: 0.38636363636363635 recall: 0.3333333333333333 precision: 0.4594594594594595 predictions: 46.0
#######0.05 SAMPLES######
F1: 0.41860465116279066 recall: 0.5294117647058824 precision: 0.34615384615384615 predictions: 68.0
#######0.075 SAMPLES######
F1: 0.43636363636363634 recall: 0.7058823529411765 precision: 0.3157894736842105 predictions: 79.0
#######0.1 SAMPLES######
F1: 0.38461538461538464 recall: 0.7843137254901961 precision: 0.25477707006369427 predictions: 75.0
#######0.125 SAMPLES######
F1: 0.3786008230452675 recall: 0.9019607843137255 precision: 0.23958333333333334 predictions: 92.0
#######0.15 SAMPLES######
F1: 0.36015325670498083 recall: 0.9215686274509803 precision: 0.22380952380952382 predictions: 81.0
#######0.175 SAMPLES######
F1: 0.3286713286713287 recall: 0.9215686274509803 precision: 0.2 predictions: 86.0
#######0.2 SAMPLES######
F1: 0.31475409836065577 recall: 0.9411764705882353 precision: 0.1889763779527559 predictions: 87.0


In [46]:
for j in sample:
    print('#######{} SAMPLES######'.format(j))

    #train_yr = train[train['year']==i]
    model = AdaBoostClassifier()
    over = SMOTE(sampling_strategy=j)
    under = RandomUnderSampler(sampling_strategy=j)
    steps = [('o', over), ('u', under)]
    pipeline = Pipeline(steps=steps)
    X, y = pipeline.fit_resample(train[adv_stats], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]
    #pred_yr.append(predictions)

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

#######0.025 SAMPLES######
F1: 0.44247787610619466 recall: 0.49019607843137253 precision: 0.4032258064516129 predictions: 62.0
#######0.05 SAMPLES######
F1: 0.4202898550724638 recall: 0.5686274509803921 precision: 0.3333333333333333 predictions: 87.0
#######0.075 SAMPLES######
F1: 0.4276729559748428 recall: 0.6666666666666666 precision: 0.3148148148148148 predictions: 108.0
#######0.1 SAMPLES######
F1: 0.3742690058479532 recall: 0.6274509803921569 precision: 0.26666666666666666 predictions: 120.0
#######0.125 SAMPLES######
F1: 0.37988826815642457 recall: 0.6666666666666666 precision: 0.265625 predictions: 128.0
#######0.15 SAMPLES######
F1: 0.37234042553191493 recall: 0.6862745098039216 precision: 0.25547445255474455 predictions: 137.0
#######0.175 SAMPLES######
F1: 0.3441860465116279 recall: 0.7254901960784313 precision: 0.22560975609756098 predictions: 164.0
#######0.2 SAMPLES######
F1: 0.3584905660377359 recall: 0.7450980392156863 precision: 0.2360248447204969 predictions: 161.0


In [47]:
for j in sample:
    print('#######{} SAMPLES######'.format(j))

    #train_yr = train[train['year']==i]
    model = AdaBoostClassifier()
    over = SMOTE(sampling_strategy=j)
    #under = RandomUnderSampler(sampling_strategy=j)
    #steps = [('o', over), ('u', under)]
    #pipeline = Pipeline(steps=steps)
    X, y = over.fit_resample(train[adv_stats], train['drafted'])
    model.fit(X, y)
    y_pred = model.predict(test_2019[adv_stats])
    predictions = [round(value) for value in y_pred]
    #pred_yr.append(predictions)

    f1 = f1_score(test_2019['drafted'], predictions)
    recall = recall_score(test_2019['drafted'], predictions)
    precision = precision_score(test_2019['drafted'], predictions)
    print('F1: {} recall: {} precision: {} predictions: {}'.format(f1, recall, precision, sum(predictions)))

#######0.025 SAMPLES######
F1: 0.42857142857142855 recall: 0.47058823529411764 precision: 0.39344262295081966 predictions: 61.0
#######0.05 SAMPLES######
F1: 0.42105263157894735 recall: 0.5490196078431373 precision: 0.34146341463414637 predictions: 82.0
#######0.075 SAMPLES######
F1: 0.40764331210191085 recall: 0.6274509803921569 precision: 0.3018867924528302 predictions: 106.0
#######0.1 SAMPLES######
F1: 0.3668639053254438 recall: 0.6078431372549019 precision: 0.2627118644067797 predictions: 118.0
#######0.125 SAMPLES######
F1: 0.4 recall: 0.6862745098039216 precision: 0.28225806451612906 predictions: 124.0
#######0.15 SAMPLES######
F1: 0.3880597014925373 recall: 0.7647058823529411 precision: 0.26 predictions: 150.0
#######0.175 SAMPLES######
F1: 0.38277511961722493 recall: 0.7843137254901961 precision: 0.25316455696202533 predictions: 158.0
#######0.2 SAMPLES######
F1: 0.36363636363636365 recall: 0.7058823529411765 precision: 0.24489795918367346 predictions: 147.0
